In [1]:
import torch
masked_state_dict = torch.load('masked_gpt2.pt')
N=100

In [2]:
from transformer_lens.HookedTransformer import HookedTransformer
gpt2_masked = HookedTransformer.from_pretrained(is_masked=True, model_name='gpt2')

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [3]:
from transformer_lens.ioi_dataset import IOIDataset

ioi_dataset = IOIDataset(prompt_type="ABBA", N=N, nb_templates=1)
train_data = ioi_dataset.toks.long()


In [4]:
def logit_diff_from_ioi_dataset(
    logits: torch.Tensor, tokens: torch.Tensor, mean=False,
):
    assert tokens.shape == (
        N,
        16,
    ), tokens.shape  # TODO check this is not breaking things...
    assert len(logits.shape) == 3, logits.shape

    io_labels = tokens[:, 2]
    s_labels = tokens[:, 4]

    io_logits = logits[torch.arange(N), -2, io_labels]
    s_logits = logits[torch.arange(N), -2, s_labels]

    logit_diff = io_logits - s_logits
    if mean:
        return logit_diff.mean()
    else:
        return logit_diff


In [5]:
gpt2_masked.load_state_dict(masked_state_dict)
logit_diff_from_ioi_dataset(gpt2_masked(train_data), train_data, mean=True)

tensor(20.6953, device='cuda:0', grad_fn=<MeanBackward0>)

In [7]:
from transformer_lens import utils
from functools import partial 

def make_forward_hooks(nodes_to_mask):
    forward_hooks = []
    for layer in range(12):
        for head in range(12):
            for qkv in ["q", "k", "v"]:
                mask_value = nodes_to_mask[f"{layer}.{head}.{qkv}"]
                def head_ablation_hook(value, hook, head_idx, layer_idx, qkv_val, mask_value):
                    value[:, :, head_idx, :] *= mask_value
                    return value

                a_hook = (utils.get_act_name(qkv, int(layer)), partial(head_ablation_hook, head_idx=head, layer_idx=layer, qkv_val=qkv, mask_value=mask_value))
                forward_hooks.append(a_hook)
    return forward_hooks


In [8]:
new_gpt2 = HookedTransformer.from_pretrained(is_masked=False, model_name='gpt2')

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [9]:
mask_value_dict = {}
for layer_index, layer in enumerate(gpt2_masked.blocks):
    for head_index in range(12):
        for q_k_v in ["q", "k", "v"]:
            # total_nodes += 1
            if q_k_v == "q":
                mask_value = (
                    layer.attn.hook_q.sample_mask()[head_index].cpu().item()
                )
            if q_k_v == "k":
                mask_value = (
                    layer.attn.hook_k.sample_mask()[head_index].cpu().item()
                )
            if q_k_v == "v":
                mask_value = (
                    layer.attn.hook_v.sample_mask()[head_index].cpu().item()
                )
            mask_value_dict[f"{layer_index}.{head_index}.{q_k_v}"] = mask_value

mask_value_dict.values()
forward_hooks = make_forward_hooks(mask_value_dict)
logit_diff_from_ioi_dataset(new_gpt2.run_with_hooks(train_data, return_type="logits", fwd_hooks=forward_hooks), train_data, mean=True)


tensor(20.4579, device='cuda:0', grad_fn=<MeanBackward0>)